In [1]:
import csv
import pandas as pd
from collections import defaultdict
from prettytable import PrettyTable
import trueskill
import altair as alt
from getpass import getpass
from apiclient.discovery import build

### Read in CSV Data

In [2]:
with open('Game Grumps VS data.csv') as csv_file:
    csv_data = [line for line in csv.reader(csv_file)]
    
    csv_df = pd.DataFrame(csv_data[1:], columns=csv_data[0])

In [3]:
vs_df = csv_df.copy()
vs_df['Loser(s)'] = vs_df['Loser(s)'].map(lambda s: [loser.strip() for loser in s.split(';')])
vs_df['Competitors'] = (vs_df['Loser(s)'] + vs_df['Winner'].map(lambda s: [s])).map(lambda l: sorted(l))

#### Add Youtube Data

In [4]:
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'
api_key = getpass('Youtube API Key')
youtube_client = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=api_key)

Youtube API Key ·······································


In [5]:
# Retrieve almost all relevant data on a given video id
# youtube_client.videos().list(part='snippet,contentDetails,statistics', id='_MIMm3OCZXA').execute()
vs_df['Publish Date'] = vs_df['Episode-Id'].map(
    lambda episode_id: youtube_client.videos().list(part='snippet', id=episode_id).execute()['items'][0]['snippet']['publishedAt']
)
vs_df = vs_df.sort_values(by='Publish Date').reset_index(drop=True)

#### Add Match Numbers

In [6]:
vs_df['Overall Match Number'] = vs_df.index + 1

In [7]:
vs_df = vs_df.join(
    vs_df.groupby(['Game', 'Episode-Id']).first()['Overall Match Number'].rename('First Game Match'),
    on=['Game', 'Episode-Id']
)
vs_df['Match Number'] = vs_df['Overall Match Number'] - vs_df['First Game Match'] + 1
vs_df = vs_df.drop(labels='First Game Match', axis=1)

In [8]:
competitor_df = vs_df.explode('Competitors').rename({'Competitors':'Competitor'}, axis=1)

In [9]:
vs_df

,Winner,Loser(s),Game,Episode-Id,Notes,Competitors,Publish Date,Overall Match Number,Match Number
0,Jon,[Arin],Super Smash Bros. Brawl,uiofcRg6XEo,Arin is red,"[Arin, Jon]",2012-08-23T01:44:05Z,1,1
1,Arin,[Jon],Super Smash Bros. Brawl,uiofcRg6XEo,,"[Arin, Jon]",2012-08-23T01:44:05Z,2,2
2,Jon,[Arin],Super Smash Bros. Brawl,uiofcRg6XEo,,"[Arin, Jon]",2012-08-23T01:44:05Z,3,3
3,Arin,[Jon],Super Smash Bros. Brawl,uiofcRg6XEo,,"[Arin, Jon]",2012-08-23T01:44:05Z,4,4
4,Arin,[Jon],Super Smash Bros. Brawl,uiofcRg6XEo,,"[Arin, Jon]",2012-08-23T01:44:05Z,5,5
...,...,...,...,...,...,...,...,...,...
152,Tie (Arin; Danny),[],Mario Golf: Toadstool Tour,wRPKjKG4KKo,,"[, Tie (Arin; Danny)]",2015-06-01T21:00:01Z,153,1
153,Danny,[Arin],Millionaire,cFurtZ7Gvb4,,"[Arin, Danny]",2016-02-05T22:00:00Z,154,1
154,Arin,"[Danny, Jacob Anderson]",Wheel of Fortune,90lejF5TzGM,,"[Arin, Danny, Jacob Anderson]",2016-04-25T17:00:00Z,155,1
155,Arin,"[Danny, Finn Wolfhard]",Wheel of Fortune,fg_QS2yAsfI,,"[Arin, Danny, Finn Wolfhard]",2017-01-15T22:00:01Z,156,1


In [10]:
competitor_df

,Winner,Loser(s),Game,Episode-Id,Notes,Competitor,Publish Date,Overall Match Number,Match Number
0,Jon,[Arin],Super Smash Bros. Brawl,uiofcRg6XEo,Arin is red,Arin,2012-08-23T01:44:05Z,1,1
0,Jon,[Arin],Super Smash Bros. Brawl,uiofcRg6XEo,Arin is red,Jon,2012-08-23T01:44:05Z,1,1
1,Arin,[Jon],Super Smash Bros. Brawl,uiofcRg6XEo,,Arin,2012-08-23T01:44:05Z,2,2
1,Arin,[Jon],Super Smash Bros. Brawl,uiofcRg6XEo,,Jon,2012-08-23T01:44:05Z,2,2
2,Jon,[Arin],Super Smash Bros. Brawl,uiofcRg6XEo,,Arin,2012-08-23T01:44:05Z,3,3
...,...,...,...,...,...,...,...,...,...
155,Arin,"[Danny, Finn Wolfhard]",Wheel of Fortune,fg_QS2yAsfI,,Arin,2017-01-15T22:00:01Z,156,1
155,Arin,"[Danny, Finn Wolfhard]",Wheel of Fortune,fg_QS2yAsfI,,Danny,2017-01-15T22:00:01Z,156,1
155,Arin,"[Danny, Finn Wolfhard]",Wheel of Fortune,fg_QS2yAsfI,,Finn Wolfhard,2017-01-15T22:00:01Z,156,1
156,Arin,[Danny],Mario Golf: Toadstool Tour,1JHX8B8m5p8,,Arin,2019-02-12T18:00:00Z,157,1


### Overview Stats

In [11]:
all_competitors = list(
    filter(
        lambda item: item and '(' not in item,
        vs_df['Winner'].append(vs_df.explode('Loser(s)')['Loser(s)']).unique()
    )
)
print(all_competitors)

['Jon', 'Arin', 'Barry', 'Danny', 'AI', 'Ross', 'Susy', 'Jacob Anderson', 'Finn Wolfhard']


In [12]:
competitor_data = {competitor: {'wins':0, 'games':0, 'won_vs':defaultdict(lambda: 0)} for competitor in all_competitors}
for i, match in vs_df.iterrows():
    winner = match['Winner']
    # TODO: add in ignoring AI in favor of top place human
    if 'AI' in winner:
        winner = 'AI'
    # TODO: Don't skip ties
    elif 'Tie' in winner:
        continue
    competitor_data[winner]['wins']+=1
    competitor_data[winner]['games']+=1
    for loser in match['Loser(s)']:
        competitor_data[loser]['games']+=1
        competitor_data[winner]['won_vs'][loser]+=1

In [38]:
table = PrettyTable(['Competitor','Wins','Games Played', 'Win Rate'])
for competitor, data in sorted(competitor_data.items()):
    win_rate = round(data['wins'] / data['games'] * 100, 2)
    table.add_row([competitor, data['wins'], data['games'], win_rate])
print(table)

+----------------+------+--------------+----------+
|   Competitor   | Wins | Games Played | Win Rate |
+----------------+------+--------------+----------+
|       AI       |  13  |      26      |   50.0   |
|      Arin      |  69  |     156      |  44.23   |
|     Barry      |  1   |      1       |  100.0   |
|     Danny      |  3   |      9       |  33.33   |
| Finn Wolfhard  |  0   |      1       |   0.0    |
| Jacob Anderson |  0   |      1       |   0.0    |
|      Jon       |  70  |     146      |  47.95   |
|      Ross      |  0   |      1       |   0.0    |
|      Susy      |  0   |      1       |   0.0    |
+----------------+------+--------------+----------+


In [14]:
table = PrettyTable(['vv Winner \ Loser ->'] + sorted(competitor_data))
for competitor in sorted(competitor_data):
    row = []
    for competitor_2 in sorted(competitor_data):
        if competitor == competitor_2:
            row.append('X')
        else:
            row.append(competitor_data[competitor]['won_vs'][competitor_2])
    table.add_row([competitor] + row)
print(table)

+----------------------+----+------+-------+-------+---------------+----------------+-----+------+------+
| vv Winner \ Loser -> | AI | Arin | Barry | Danny | Finn Wolfhard | Jacob Anderson | Jon | Ross | Susy |
+----------------------+----+------+-------+-------+---------------+----------------+-----+------+------+
|          AI          | X  |  13  |   0   |   1   |       0       |       0        |  12 |  0   |  0   |
|         Arin         | 5  |  X   |   0   |   5   |       1       |       1        |  64 |  0   |  0   |
|        Barry         | 0  |  1   |   X   |   0   |       0       |       0        |  0  |  1   |  1   |
|        Danny         | 0  |  3   |   0   |   X   |       0       |       0        |  0  |  0   |  0   |
|    Finn Wolfhard     | 0  |  0   |   0   |   0   |       X       |       0        |  0  |  0   |  0   |
|    Jacob Anderson    | 0  |  0   |   0   |   0   |       0       |       X        |  0  |  0   |  0   |
|         Jon          | 8  |  70  |   0   |  

### TrueSkill Analysis

In [15]:
competitor_rankings = defaultdict(trueskill.Rating)
competitor_ranking_over_time = []

for _, match in vs_df.iterrows():
    if 'AI' in match['Winner']:
        # If the AI wins we mark it as rank 0 (the best), the human that was was closest to first as rank 1
        # and all other humans as rank 2
        top_human = match['Winner'][match['Winner'].index('(')+1:-1]
        losers = [competitor for competitor in match['Loser(s)'] if competitor != top_human]
        competitors = [[competitor_rankings['AI']], [competitor_rankings[top_human]]]
        competitors += [[competitor_rankings[competitor]] for competitor in losers]
        rankings = [0, 1] + [2 for competitor in losers]
        match_result = trueskill.rate(competitors, rankings)
        # We pass the AI first, then top_human, then others
        winner_ranking = match_result[0][0]
        competitor_rankings['AI'] = winner_ranking
        competitor_ranking_over_time.append({
            'overall_match_number': match['Overall Match Number'],
            'competitor': 'AI',
            'mu': winner_ranking.mu,
            'sigma': winner_ranking.sigma
        })
        top_human_ranking = match_result[1][0]
        competitor_rankings[top_human] = top_human_ranking
        competitor_ranking_over_time.append({
            'overall_match_number': match['Overall Match Number'],
            'competitor': top_human,
            'mu': top_human_ranking.mu,
            'sigma': top_human_ranking.sigma
        })
        for i, ranking in enumerate(match_result[2:]):
            loser = losers[i]
            # Other rankings and 'Loser(s)' share order
            competitor_rankings[loser] = ranking[0]
            competitor_ranking_over_time.append({
            'overall_match_number': match['Overall Match Number'],
            'competitor': loser,
            'mu': ranking[0].mu,
            'sigma': ranking[0].sigma
        })
    elif 'Tie' in match['Winner']:
        # When there is a tie for the winners, all of those tied in first get rank 0 (the best), and everyone else gets rank 1
        tied_competitors = match['Winner'][match['Winner'].index('(')+1:-1].split('; ')
        losers = [competitor for competitor in match['Loser(s)'] if competitor and competitor not in tied_competitors]
        competitors = [[competitor_rankings[competitor]] for competitor in tied_competitors]
        competitors += [[competitor_rankings[competitor]] for competitor in losers]
        rankings = [0 for competitor in tied_competitors] + [1 for competitor in losers]
        match_result = trueskill.rate(competitors, rankings)
        # We put the tied parties first, then everyone else
        for i, ranking in enumerate(match_result[:len(tied_competitors)]):
            winner = tied_competitors[i]
            # Other rankings and 'Loser(s)' share order
            competitor_rankings[winner] = ranking[0]
            competitor_ranking_over_time.append({
                'overall_match_number': match['Overall Match Number'],
                'competitor': winner,
                'mu': ranking[0].mu,
                'sigma': ranking[0].sigma
            })
        for i, ranking in enumerate(match_result[len(tied_competitors):]):
            loser = losers[i]
            # Other rankings and 'Loser(s)' share order
            competitor_rankings[loser] = ranking[0]
            competitor_ranking_over_time.append({
                'overall_match_number': match['Overall Match Number'],
                'competitor': loser,
                'mu': ranking[0].mu,
                'sigma': ranking[0].sigma
            })
    else:
        # Regular Game Grumps VS case
        # Due to how I collected data, we rank the winner as rank 0 and all other competitors as rank 1, teams are all 1-player
        # This also allows us to ignore cases where multiple AI have different ranks in the same match
        competitors = [[competitor_rankings[match['Winner']]]]
        competitors += [[competitor_rankings[competitor]] for competitor in match['Loser(s)']]
        rankings = [0] + [1 for competitor in match['Loser(s)']]
        match_result = trueskill.rate(competitors, rankings)
        # We pass the winner in first, and these are one-player teams
        winner_ranking = match_result[0][0]
        competitor_rankings[match['Winner']] = winner_ranking
        competitor_ranking_over_time.append({
            'overall_match_number': match['Overall Match Number'],
            'competitor': match['Winner'],
            'mu': winner_ranking.mu,
            'sigma': winner_ranking.sigma
        })
        for i, ranking in enumerate(match_result[1:]):
            loser = match['Loser(s)'][i]
            # Other rankings and 'Loser(s)' share order
            competitor_rankings[loser] = ranking[0]
            competitor_ranking_over_time.append({
            'overall_match_number': match['Overall Match Number'],
            'competitor': loser,
            'mu': ranking[0].mu,
            'sigma': ranking[0].sigma
        })
        
print(dict(competitor_rankings))
competitor_ranking_over_time_df = pd.DataFrame(competitor_ranking_over_time)
competitor_ranking_over_time_df
            

{'Jon': trueskill.Rating(mu=24.767, sigma=0.791), 'Arin': trueskill.Rating(mu=25.096, sigma=0.787), 'AI': trueskill.Rating(mu=26.854, sigma=1.222), 'Barry': trueskill.Rating(mu=30.835, sigma=6.161), 'Ross': trueskill.Rating(mu=24.057, sigma=4.628), 'Susy': trueskill.Rating(mu=24.058, sigma=4.637), 'Danny': trueskill.Rating(mu=23.054, sigma=2.139), 'Jacob Anderson': trueskill.Rating(mu=22.763, sigma=4.676), 'Finn Wolfhard': trueskill.Rating(mu=22.520, sigma=4.673)}


,overall_match_number,competitor,mu,sigma
0,1,Jon,29.395832,7.171476
1,1,Arin,20.604168,7.171476
2,2,Arin,26.643243,6.040360
3,2,Jon,23.356757,6.040360
4,3,Jon,27.084742,5.261314
...,...,...,...,...
339,156,Arin,25.023540,0.786509
340,156,Danny,23.625593,2.222543
341,156,Finn Wolfhard,22.520280,4.673472
342,157,Arin,25.095787,0.787136


In [40]:
table = PrettyTable(['Competitor', 'Final Skill Level'])
for competitor, rating in sorted(competitor_rankings.items()):
    table.add_row([competitor, round(rating.mu, 2)])
print(table)

+----------------+-------------------+
|   Competitor   | Final Skill Level |
+----------------+-------------------+
|       AI       |       26.85       |
|      Arin      |        25.1       |
|     Barry      |       30.84       |
|     Danny      |       23.05       |
| Finn Wolfhard  |       22.52       |
| Jacob Anderson |       22.76       |
|      Jon       |       24.77       |
|      Ross      |       24.06       |
|      Susy      |       24.06       |
+----------------+-------------------+


In [16]:
# Use this cell if you want to plot a pre-match starting point for trueskill
add_pre_start = True
if add_pre_start:
    first_comp_df = competitor_ranking_over_time_df.groupby('competitor').min()
    first_comp_df['competitor'] = first_comp_df.index
    first_comp_df['overall_match_number'] = first_comp_df['overall_match_number'] - 1
    starting_mu, starting_sigma = trueskill.Rating()
    first_comp_df['mu'] = starting_mu
    first_comp_df['sigma'] = starting_sigma
    competitor_ranking_over_time_df = competitor_ranking_over_time_df.append(first_comp_df).sort_values('overall_match_number').reset_index()
    competitor_ranking_over_time_df

In [20]:
relative_match_df = pd.DataFrame()
for competitor in all_competitors:
    comp_df = competitor_ranking_over_time_df[competitor_ranking_over_time_df['competitor'] == competitor].reset_index(drop=True)
    comp_df['competitor_match_number'] = comp_df.index + (0 if add_pre_start else 1)
    relative_match_df = relative_match_df.append(comp_df)
competitor_ranking_over_time_df = relative_match_df
competitor_ranking_over_time_df

,index,overall_match_number,competitor,mu,sigma,competitor_match_number
0,Jon,0,Jon,25.000000,8.333333,0
1,0,1,Jon,29.395832,7.171476,1
2,3,2,Jon,23.356757,6.040360,2
3,4,3,Jon,27.084742,5.261314,3
4,7,4,Jon,23.719288,4.630351,4
...,...,...,...,...,...,...
1,319,147,Susy,24.058497,4.636977,1
0,Jacob Anderson,154,Jacob Anderson,25.000000,8.333333,0
1,338,155,Jacob Anderson,22.762948,4.676397,1
0,Finn Wolfhard,155,Finn Wolfhard,25.000000,8.333333,0


In [35]:
color_order = list(competitor_ranking_over_time_df.groupby('competitor').min().sort_values('overall_match_number').index)

In [33]:
base = alt.Chart(competitor_ranking_over_time_df, title='Skill Level over Time').encode(
    x=alt.X('overall_match_number', title='Overall Match Number'),
    color=alt.Color('competitor', title='Competitor', sort=color_order),
)

legend_selection = alt.selection_multi(fields=['competitor'], bind='legend')
match_slider = alt.binding_range(min=1, max=competitor_ranking_over_time_df['overall_match_number'].max(), step=1, name='Final Match Number:')
match_selection = alt.selection_single(name="final_match_number", fields=['overall_match_number'],
                                   bind=match_slider, init={'overall_match_number':competitor_ranking_over_time_df['overall_match_number'].max()})

mu = base.mark_line(strokeWidth=3).encode(
    y=alt.Y('mu', title='Skill Level', scale=alt.Scale(zero=False)),
    opacity=alt.condition(legend_selection, alt.value(1), alt.value(0.1))
)

sigma = base.mark_area().encode(
    y='y:Q',
    y2='y2:Q',
    fillOpacity=alt.condition(legend_selection, alt.value(0.3), alt.value(0.05))
).transform_calculate(
    y='datum.mu+datum.sigma',
    y2='datum.mu-datum.sigma'
)

(mu + sigma).add_selection(
    legend_selection
).add_selection(
    match_selection
).transform_filter(
    alt.datum.overall_match_number <= match_selection.overall_match_number
).properties(
    width=800,
    height=500
)

alt.LayerChart(...)

In [34]:
base = alt.Chart(competitor_ranking_over_time_df, title='Skill Level over Time').encode(
    x=alt.X('competitor_match_number', title='Competitor Match Number'),
    color=alt.Color('competitor', title='Competitor', sort=color_order),
)

legend_selection = alt.selection_multi(fields=['competitor'], bind='legend')
match_slider = alt.binding_range(min=1, max=competitor_ranking_over_time_df['competitor_match_number'].max(), step=1, name='Final Match Number:')
match_selection = alt.selection_single(name="final_match_number", fields=['competitor_match_number'],
                                   bind=match_slider, init={'competitor_match_number':competitor_ranking_over_time_df['competitor_match_number'].max()})

mu = base.mark_line(strokeWidth=3).encode(
    y=alt.Y('mu', title='Skill Level', scale=alt.Scale(zero=False)),
    opacity=alt.condition(legend_selection, alt.value(1), alt.value(0.1))
)

sigma = base.mark_area().encode(
    y='y:Q',
    y2='y2:Q',
    fillOpacity=alt.condition(legend_selection, alt.value(0.3), alt.value(0.05))
).transform_calculate(
    y='datum.mu+datum.sigma',
    y2='datum.mu-datum.sigma'
)

(mu + sigma).add_selection(
    legend_selection
).add_selection(
    match_selection
).transform_filter(
    alt.datum.competitor_match_number <= match_selection.competitor_match_number
).properties(
    width=800,
    height=500
)

alt.LayerChart(...)